**Пайплайн:**

- ✅ Сначала нам надо скачать дату – соберите как минимум 60 (30 положительных и 30 отрицательных) отзывов на похожие продукты (не надо мешать отзывы на отели с отзывами на ноутбуки) для составления " тонального словаря" (чем больше отзывов, тем лучше) и 10 отзывов для проверки качества.


- ✅ Токенизируйте слова, приведите их к нижнему регистру и к начальной форме (1 балл за токенизацию, 1 - за начальную форму)


- ✅ Составьте 2 множества - в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Попробуйте поиграть с частотностями и исключить шум (к примеру, выбросить слова, встречающиеся 1-2 раза) (3 балла) (если у вас получились пустые множества, уберите фильтр по частотности или увеличьте выборку)


- ✅ Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy (1 - за коректно работающую функцию, 1 - за подсчёт accuracy)


- Предложите как минимум 2 способа улучшить эту программу с помощью добавления к ней любых мулек - просто словами, писать улучшающий код не надо (1 балл)

# №1 Parsing data

Cобираем >60 (30 положительных и 30 отрицательных) отзывов на похожие продукты  для составления "тонального словаря" и 10 отзывов для проверки качества.

**Берём отзывы на рязанские отели с сайта tripadvisor.ru**

In [69]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from sklearn.metrics import accuracy_score

from collections import Counter
from pymystem3 import Mystem
import pandas as pd
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize

from tqdm import tqdm
import time
import csv
import os
import re

m = Mystem()
pymorph = pymorphy2.MorphAnalyzer()
punctuation = """!"#$%&'()*+, -./:;<=>?@[\]^_`{|}~–—«»"""

In [20]:
firefoxdriver_path = '/Users/mariabocharova/PycharmProjects/REALEC/firefoxdriver'
chromedriver_path = '/Users/mariabocharova/PycharmProjects/REALEC/chromedriver'

hotels = ['https://www.tripadvisor.ru/Hotel_Review-g298485-d1732311-Reviews-Congress_Hotel_Forum-Ryazan_Ryazan_Oblast_Central_Russia.html', 
          'https://www.tripadvisor.ru/Hotel_Review-g15512259-d6533471-Reviews-Tourist_Base_Polushkino-Polushkino_Ryazan_Oblast_Central_Russia.html', 
          'https://www.tripadvisor.ru/Hotel_Review-g298485-d7257464-Reviews-Hostel_Like-Ryazan_Ryazan_Oblast_Central_Russia.html', 
          'https://www.tripadvisor.ru/Hotel_Review-g298485-d10817084-Reviews-Kremlyovskiy-Ryazan_Ryazan_Oblast_Central_Russia.html', 
          'https://www.tripadvisor.ru/Hotel_Review-g298485-d1469261-Reviews-Aragon_Hotel-Ryazan_Ryazan_Oblast_Central_Russia.html',
          'https://www.tripadvisor.ru/Hotel_Review-g298485-d5534355-Reviews-V_Nekotorom_Tsarstve_Hotel-Ryazan_Ryazan_Oblast_Central_Russia.html',
          'https://www.tripadvisor.ru/Hotel_Review-g298485-d7133243-Reviews-Hotel_Ryazan-Ryazan_Ryazan_Oblast_Central_Russia.html']

In [3]:
df_reviews = pd.DataFrame(columns=['positive', 'negative'])

In [4]:
def get_reviews(hotel_link, save_to_path, firefoxdriver_path='/Users/mariabocharova/PycharmProjects/REALEC/firefoxdriver'):
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}')
    driver = webdriver.Firefox(executable_path=firefoxdriver_path)
    driver.get(hotel_link)
    time.sleep(5)

    with open(save_to_path, 'a') as csvfile:
        writer = csv.writer(csvfile)
        try:
            while driver.find_element_by_xpath(".//a[@class='ui_button nav next primary ']"):
                # Нахожу блок с оценкой и отзывом
                reviews = driver.find_elements_by_xpath("//div[@data-test-target='HR_CC_CARD']")
                for review in reviews:
                    text = review.find_element_by_xpath(".//q[@class='XllAv H4 _a']").text
                    score = float(review.find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]")
                                .get_attribute("class").strip('ui_bubble_rating bubble_'))
                    # Сохраняю оценку и комментарий
                    writer.writerow([score / 10, text])
                driver.find_element_by_xpath(".//a[@class='ui_button nav next primary ']").click()
                time.sleep(7)
        except Exception as e:
            pass
    driver.close()

In [ ]:
with open('tripadvisor_reviews.csv', 'a') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['score', 'text'])
for link in tqdm(hotels):
    get_reviews(link, 'tripadvisor_reviews.csv')

**Всего отзывов:**

In [21]:
df = pd.read_csv('tripadvisor_reviews.csv')
len(df)

615

Будем считать, что отзыв является положительным, если его оценивают на 4-5. Если оценка меньше – то отзыв отрицательный. 

In [22]:
df['score'].value_counts(sort=True)

5.0    309
4.0    183
3.0     72
1.0     29
2.0     22
Name: score, dtype: int64

Выберем рандомные 120 положительных и 120 отрицательных отзывов. 

In [23]:
df_positive = df.loc[df['score'] >= 4].sample(n = 120)
df_negative = df.loc[df['score'] < 4].sample(n = 120)

**Достанем отдельные отзывы для предсказания**

In [ ]:
link = 'https://www.tripadvisor.ru/Hotel_Review-g298485-d6851547-Reviews-Esenin-Ryazan_Ryazan_Oblast_Central_Russia.html'
get_reviews(link, 'tripadvisor_reviews_prediction.csv')

In [26]:
def lemmatize(text):
    lemm_text = ''
    for word in m.lemmatize(text):
        if word.isalpha():
            lemm_text += f'{word} '
    return lemm_text

In [36]:
with open('tripadvisor_reviews_prediction.csv', encoding='utf-8') as f:
    next(f)
    negative_predict = [lemmatize(line[5:-2]).strip() for line in f if float(line[0:3]) < 4][:10]

with open('tripadvisor_reviews_prediction.csv', encoding='utf-8') as f:
    next(f)
    positive_predict = [lemmatize(line[5:-2]).strip() for line in f if float(line[0:3]) >= 4][:10]      

In [37]:
df_predict = pd.read_csv('tripadvisor_reviews_prediction.csv')
df_predict['score'].value_counts(sort = True)

4.0    34
5.0    32
3.0    28
2.0    11
1.0    10
Name: score, dtype: int64

# №2 Токенизация

Токенизируем слова, приводим их к нижнему регистру и к начальной форме.

In [38]:
df_positive['text_lemm'] = df_positive['text'].apply(lemmatize)
df_negative['text_lemm'] = df_negative['text'].apply(lemmatize)

In [39]:
df_positive.head(3)

,score,text,text_lemm
198,5.0,Остановились на 1 ночь. Номер семейный оказалс...,останавливаться на ночь номер семейный оказыва...
294,4.0,Средний отель) большой плюс - это расположение...,средний отель большой плюс этот расположение о...
538,4.0,На первый взгляд простая кирпичная коробка оте...,на первый взгляд простой кирпичный коробка оте...


# №3 Множества

Составляем 2 множества – в одном  слова, которые встречаются только в положительных отзывах, а в другом – встречающиеся только в отрицательных. 

In [106]:
def make_sets_of_words(df_positive, df_negative):
    positive_reviews = Counter(' '.join(df_positive['text_lemm']).split())
    negative_reviews = Counter(' '.join(df_negative['text_lemm']).split())

    positive_reviews = set({word: n for word, n in positive_reviews.items() if n > 1})
    negative_reviews = set({word: n for word, n in negative_reviews.items() if n > 1})

    positive_words_set = positive_reviews - negative_reviews
    negative_words_set = negative_reviews - positive_reviews
    
    return positive_words_set, negative_words_set

# №4 Предсказание

Создаем функцию, которая будет определять, положительный отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и считаем качество при помощи accuracy (1 - за коректно работающую функцию, 1 - за подсчёт accuracy)

In [126]:
def predict_tone(text, positive_words_set, negative_words_set):
    text = lemmatize(text).split()
    bigrams = list(nltk.bigrams(text))

    positive_unigram_num = len([word for word in text if word in positive_words_set])
    negative_unigram_num = len([word for word in text if word in negative_words_set])
    
    # Добавила вот эти строки
    positive_bigram_num = len([bigram for bigram in bigrams if ' '.join(bigram) in positive_words_set])
    negative_bigram_num = len([bigram for bigram in bigrams if ' '.join(bigram) in negative_words_set])
    
    positive_percent = round((positive_unigram_num + positive_bigram_num) / len(text) * 100, 2)
    negative_percent = round((negative_unigram_num + negative_bigram_num) / len(text) * 100, 2)
    
    return positive_percent, negative_percent

**Считаем accuracy для функции определения негативных отзывов**

In [108]:
neg_gold = [1] * 10
neg_results = []

for prediction in negative_predict:
    positive_words_set, negative_words_set = make_sets_of_words(df_positive, df_negative)
    pos, neg = predict_tone(prediction, positive_words_set, negative_words_set)
    if neg > pos:
        neg_results.append(1)
    else:
        neg_results.append(0)
print('Accuracy of detecting negative reviews:', accuracy_score(neg_gold, neg_results))

Accuracy of detecting negative reviews: 0.9


**Считаем accuracy для функции определения положительных отзывов**

In [109]:
pos_gold = [1] * 10
pos_results = []

for prediction in positive_predict:
    positive_words_set, negative_words_set = create_sets_of_words(df_positive, df_negative)
    pos, neg = predict_tone(prediction, positive_words_set, negative_words_set)
    if pos > neg:
        pos_results.append(1)
    else:
        pos_results.append(0)
print('Accuracy of detecting positive reviews:', accuracy_score(pos_gold, pos_results))

Accuracy of detecting positive reviews: 0.2


# Limitations, или как попробовать улучшить 

1. Можно попробовать использовать сеты из биграмм, а не из единичных терминов. Тогда, например, сеты составленные из отзывов _мне не понравился отель_ и _мне понравился отель_ будут отличаться не только наличием/отсутствием частицы _не_, по которой нельзя понять, к чему она относится. 
2. Можно добавить POS tags и учитывать при анализе только Noun, Adjective, Adverb, Verb.


# UPD после второй домашки

In [18]:
def tokenizer(text):
    return [i for i in nltk.word_tokenize(text) if i not in punctuation]

In [15]:
def get_pattern_NEG_VERB(tokens):
    pattern_matches = []
    
    for i in range(len(tokens)-1):
        current_token = tokens[i].lower()
        next_token = tokens[i+1].lower()
        if current_token == 'не' and pymorph.parse(next_token)[0].tag.POS in ['INFN', 'GRND', 'PRTF', 'PRTS']:
            pattern_matches.append((current_token, next_token))
    
    return pattern_matches 

In [16]:
def get_pattern_ADV_VERB(tokens):
    pattern_matches = []

    for i in range(len(tokens)-2):
        current_token = tokens[i].lower()
        next_token = tokens[i+1].lower()
        
        current_token_POS = pymorph.parse(current_token)[0].tag.POS
        next_token_POS = pymorph.parse(next_token)[0].tag.POS
        
        if current_token_POS == 'ADVB' and next_token_POS == 'VERB':
            pattern_matches.append((current_token, next_token))
    return pattern_matches

In [17]:
def get_pattern_ADJ_NOUN(tokens):
    pattern_matches = []

    for i in range(len(tokens)-1):
        current_token = tokens[i].lower()
        next_token = tokens[i+1].lower()
        current_token_POS = pymorph.parse(current_token)[0].tag.POS
        next_token_POS = pymorph.parse(next_token)[0].tag.POS
        if (current_token_POS == 'ADJF' or current_token_POS == 'ADJS') and next_token_POS == 'NOUN':
            pattern_matches.append((current_token, next_token))
    return pattern_matches

**План действий:**
1. Делаю сеты из положительных/отрицательных униграм.
2. Делаю сеты положительных/отрицательных биграмм, которые соответствуют шаблонам. 
3. Проверяю тестовый текст:
    - лемматизирую и считаю пересечения с двумя сетами униграмм. 
    - делаю POS-анализ, и для полученных биграмм ищу пересечения с сетами биграмм.

### 1. Делаю функцию для создания множеств из паттерн биграмм

In [71]:
positive_texts = ' '.join(df_positive['text'])
negative_texts = ' '.join(df_negative['text'])  

In [99]:
def make_set_of_patterns(texts):
    pattern_set = set()
    for nv in get_pattern_NEG_VERB(tokenizer(texts)):
        nv = [pymorph.parse(nv[0])[0].normal_form, pymorph.parse(nv[1])[0].normal_form]
        pattern_set.add(' '.join(nv))

    for av in get_pattern_ADV_VERB(tokenizer(texts)):
        av = [pymorph.parse(av[0])[0].normal_form, pymorph.parse(av[1])[0].normal_form]
        pattern_set.add(' '.join(av))

    for an in get_pattern_ADJ_NOUN(tokenizer(texts)):
        an = [pymorph.parse(an[0])[0].normal_form, pymorph.parse(an[1])[0].normal_form]
        pattern_set.add(' '.join(an))
    
    return pattern_set

In [104]:
positive_patterns_full_set = make_set_of_patterns(positive_texts)
negative_patterns_full_set = make_set_of_patterns(negative_texts)

positive_patterns_set = positive_patterns_full_set - negative_patterns_full_set
negative_patterns_set = negative_patterns_full_set - positive_patterns_full_set

In [115]:
positive_full_set = positive_words_set | positive_patterns_set
negative_full_set = negative_words_set | negative_patterns_set

In [123]:
neg_score = 0
for prediction in negative_predict:
    pos, neg = predict_tone(prediction, positive_full_set, negative_full_set)
    if neg > pos:
        neg_score += 1

print('Accuracy of detecting negative reviews:', neg_score / len(negative_predict))

Accuracy of detecting negative reviews: 0.9


In [124]:
pos_score = 0
for prediction in positive_predict:
    pos, neg = predict_tone(prediction, positive_full_set, negative_full_set)
    if pos > neg:
        pos_score += 1
print('Accuracy of detecting positive reviews:', pos_score / len(positive_predict))

Accuracy of detecting positive reviews: 0.4


## Итог: увеличилось accuracy of detecting positive reviews: 0.4 🌟!